In [6]:
import pandas as pd
import re
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from keras.models import load_model

In [7]:
def preprocessing(x):
    x = x.lower()
    x = re.sub('\[(.*?)\]', '', x)
    x = re.sub('[\n]{2,}', '\n',x)
    x = re.sub("[^a-z\'\s\n]",'',x)
    x = re.sub('[\n]', ' *newline* ',x)
    x = re.sub('^\s','',x)
    x = re.sub('[\s]{2,}', ' ',x)
    out =np.array( x.split(' ') )
    out = out[out!= '']
    return out 

In [8]:
index_to_word =  pickle.load( open( "index_to_word.p", "rb" ) )
word_to_index =  pickle.load( open( "word_to_index.p", "rb" ) )

In [9]:
model_load = load_model('mAIchael_buble')

def make_lyrics( input, n_words):
    input_p = preprocessing(input)
    for i in range(n_words):
        input_pad = pad_sequences( [[word_to_index[word] if word in word_to_index.keys() else 0  for word in input_p ]] , maxlen = 30, padding='pre')
        predict = model_load.predict_classes(input_pad, verbose=0)[0]
        if index_to_word[predict] == '*newline*':
            input = input + '\n'
        else:
            input = input +' '+ index_to_word[predict] 
        input_p = np.append(input_p, index_to_word[predict])
    return input

In [10]:
input  = '''may this Christmases'''

print(make_lyrics( input, 10))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
may this Christmases the fun we had last year
 christmas pretty lights


In [11]:
from flask import Flask, request, render_template

app = Flask(__name__)

@app.route('/')
def my_form():
    method = request.method
    return render_template('index.html', method= method)

@app.route('/', methods=['POST'])
def my_form_post():
    text = request.form['text']
    processed_text =  make_lyrics( text, 100)
    
    song = processed_text.replace('\n', '<br>')
    method = request.method
    
    return render_template('index.html', method= method, song = song)


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Dec/2020 21:42:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2020 21:42:56] "POST / HTTP/1.1" 200 -
